In [9]:
# Mounting Google drive used for this code.
# The inputs and outputs to the code are stored in the Google drive
from google.colab import drive
drive.mount('/content/gdrive')

# Import files
import numpy as np
from PIL import Image
import datetime
import pandas as pd
import tensorflow as tf
import math
import statistics
from scipy.stats.stats import pearsonr
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import random
import datetime

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
# Paths for data and outputs
path_PPT = '/content/gdrive/My Drive/Mixil_work/D-SHIELD/Codes/Inputs/SMAP_PPT_global/'
path_SM = '/content/gdrive/My Drive/Mixil_work/D-SHIELD/Codes/Inputs/SMAP_SM_global_ass_Oct/'
path_OP = "/content/gdrive/My Drive/Mixil_work/D-SHIELD/Codes/Outputs/Uncertainty_variation_exp_v2/4_Oct/run6/"

# Lists to store RMSE and bias calculated agaisnt the true data (SMAP L4)
RMSE = [] 
bias = []

# The dates can be changed, this is coded to take the previous three I/P files from "start" datetime to forecast till "end" datetime
for i_date in range(4,6):
  start = datetime.datetime(2020,10,i_date,16,30,0) # START DATETIME
  end = datetime.datetime(2020,10,i_date+3,22,30,0) # END DATETIME
  delta = datetime.timedelta(hours=3) # DELTA T. For SMAP L4 DELTA=3hours

  frames_PPT = [] 
  frames_SM = []

  # while loop to collect all the SM and PPT data for the specified dates.
  # Note that in real-time running, only the antecedent I/Ps are required.
  # To compare with actual SMAP, all the dates are collected.
  while start <= end:
    d = start.strftime("%Y%m%d%H%M%S")
    img_PPT = Image.open(path_PPT+d+"_ppt.jpeg")# Importing SMAP precipitation data

    # Image preprocessing
    f_PPT = img_PPT.convert(mode='RGB')
    f_PPT = np.array(f_PPT)
    f_PPT = np.sum(f_PPT, axis=2)
    f_PPT = np.array(f_PPT)
    f_PPT = f_PPT.reshape(f_PPT.shape[0], f_PPT.shape[1], 1)
    frames_PPT.append(f_PPT)
    
    img_SM = Image.open(path_SM + d +"_sm.jpeg")# Importing SMAP soil moisture data

    # Image preprocessing
    f_SM = img_SM.convert(mode='RGB')
    f_SM = np.array(f_SM)
    f_SM = np.sum(f_SM, axis=2)
    f_SM = np.array(f_SM)
    f_SM = f_SM.reshape(f_SM.shape[0], f_SM.shape[1], 1)
    frames_SM.append(f_SM)

    start += delta

  frames_PPT = np.array(frames_PPT)
  frames_SM = np.array(frames_SM)
  print(f"Shape of frames_PPT is {np.shape(frames_PPT)}")
  print(f"shape of frames_SM is {np.shape(frames_SM)}")

  # Max-Min Normalization of the SM and PPT images

  test_frames_SM = frames_SM[0:(len(frames_SM)),:,:,:]
  test_frames_PPT = frames_PPT[0:(len(frames_PPT)),:,:,:]

  maxi_SM = np.max(test_frames_SM)
  mini_SM = np.min(test_frames_SM)

  maxi_PPT = np.max(test_frames_PPT)
  mini_PPT = np.min(test_frames_PPT)

  test_frames_SM = (test_frames_SM-mini_SM)/(maxi_SM-mini_SM)
  test_frames_PPT = (test_frames_PPT-mini_PPT)/(maxi_PPT-mini_PPT)

  print(f"Shape of test_frames_SM is {np.shape(test_frames_SM)}")
  print(f"Shape of test_frames_PPT is {np.shape(test_frames_PPT)}")

  window = 3 # Interval, using 9 hours interval(i.e., three data prediction)

  # The window size is a variable. Slight modifications to the code is required if window size is changed

  # Appending 3 images to a single variable ---> "input to the model" data structure
  X_test_SM, X_test_PPT, y_act = [], [], []

  for i in range(0, len(test_frames_PPT), 3):
    X_test_SM.append(test_frames_SM[i:i+window,::,::,::])
    X_test_PPT.append(test_frames_PPT[i:i+window,::,::,::])

  for i in range(0, len(test_frames_SM), 3):
    y_act.append(test_frames_SM[i:i+window,::,::,::])
    
  X_test_SM = np.array(X_test_SM)
  X_test_PPT = np.array(X_test_PPT)
  y_act = np.array(y_act)

  print(f"Shape of X_test_SM is {np.shape(X_test_SM)}")
  print(f"Shape of X_test_PPT is {np.shape(X_test_PPT)}")
  print(f"Shape of y_act is {np.shape(y_act)}")

  # Loading trained model

  model = tf.keras.models.load_model('/content/gdrive/My Drive/Mixil_work/D-SHIELD/Codes/Models/global_model_v1.h5')

  # Prediction

  # Lists to store intermediate average RMSE and bias of the predictions
  avg_RMSE = []
  avg_bias = []
  
  # a and b are just dummy variables, since np.newaxis did not work in a single line
  # xi's are the inputs, other inputs can be added

  a = X_test_SM[0,::,::,::,::] 
  x1 = a[np.newaxis,::,::,::,::] # x1 is the input - SM

  dfs = []

  # Predicting for three days
  for i in range(1,9):
    b = X_test_PPT[i-1,::,::,::,::]
    x2 = b[np.newaxis,::,::,::,::] # x2 is the input - PPT
    
    # Predicting in three hour intervals, model trained that way
    for j in range(0,3):
      predictions = []
      n_iter = 2 # Number of iterations for prediction, using dropout layer to predict the mean and other statistics
      for t in range(n_iter):
        pred = model.predict([x1, x2])
        y_pred = pred[::,j,::,::,::].flatten()
        predictions.append(y_pred)
      
      prediction_df = pd.DataFrame()
      pred_array = np.array(predictions)
      prediction_df['mean'] = pred_array.mean(axis=0).reshape(-1,)
      prediction_df['std'] = pred_array.std(axis=0).reshape(-1,)
      prediction_df['var'] = pred_array.var(axis=0).reshape(-1,)

      y_true = y_act[i,j,::,::,::].flatten() # Flattened image as CSV

      pred_mean = prediction_df['mean']
      std = prediction_df['std']
      var = prediction_df['var']

      dict_tosave = {"y_pred":pred_mean,
                    "Std_dev":std,
                    "Var":var}
      
      df_tosave = pd.DataFrame(dict_tosave)
      dfs.append(df_tosave)
      
      avg_RMSE.append(math.sqrt(mean_squared_error(y_true, pred_mean)))
      avg_bias.append(abs(statistics.mean(pred_mean)-statistics.mean(y_true)))
      
      
    a = pred[0,::,::,::,::]
    x1 = a[np.newaxis,::,::,::,::]

  # Saving predictions (Note to change the datetime and also the required predictions to be saved)
  pred_start = datetime.datetime(2020,10,i_date,1,30,0)
  pred_end = datetime.datetime(2020,10,i_date+3,22,30,0)
  k = np.shape(dfs)[0]-1
  
  # Saving from last prediction
  while pred_end >= pred_start:
    pred_d = pred_end.strftime("%Y%m%d%H%M%S")
    dfs[k].to_csv(path_OP+pred_d+".csv",index=False)
    k = k-1
    pred_end -= delta

  RMSE = RMSE+avg_RMSE[-8:]
  bias = bias+avg_bias[-8:]

  print("ONE DAY DONE!")

# The RMSE and bias are stored as "Statistics.csv" file 
stat_dict = {"RMSE": RMSE,
             "Bias": bias}
stat_df = pd.DataFrame(stat_dict)
stat_df.to_csv(path_OP+"Statistics.csv",index=False)

Shape of frames_PPT is (27, 1624, 3856, 1)
shape of frames_SM is (27, 1624, 3856, 1)
Shape of test_frames_SM is (27, 1624, 3856, 1)
Shape of test_frames_PPT is (27, 1624, 3856, 1)
Shape of X_test_SM is (9, 3, 1624, 3856, 1)
Shape of X_test_PPT is (9, 3, 1624, 3856, 1)
Shape of y_act is (9, 3, 1624, 3856, 1)
1/1 [==============================] - 0s 79ms/step
ONE DAY DONE!
Shape of frames_PPT is (27, 1624, 3856, 1)
shape of frames_SM is (27, 1624, 3856, 1)
Shape of test_frames_SM is (27, 1624, 3856, 1)
Shape of test_frames_PPT is (27, 1624, 3856, 1)
Shape of X_test_SM is (9, 3, 1624, 3856, 1)
Shape of X_test_PPT is (9, 3, 1624, 3856, 1)
Shape of y_act is (9, 3, 1624, 3856, 1)
1/1 [==============================] - 0s 84ms/step
ONE DAY DONE!


In [ ]:
# No assimilation run

# Paths for data and outputs
path_PPT = '/content/gdrive/My Drive/Archana/Colab_Notebooks/Data/SMAP_PPT_global/'
path_SM = '/content/gdrive/My Drive/Archana/Colab_Notebooks/Data/SMAP_SM_global/'
path_OP = "/content/gdrive/My Drive/Archana/Colab_Notebooks/Outputs/Uncertainty_reduction_experiment_4_Oct/Pred_0/"

RMSE = []
bias = []

# For loop for predicting every day for three days into the future using the last three(16:30,19:30:22:30) data
for i_date in range(1,3):
  start = datetime.datetime(2020,10,i_date,16,30,0) # START DATETIME
  end = datetime.datetime(2020,10,i_date+3,22,30,0) # END DATETIME
  delta = datetime.timedelta(hours=3) # DELTA T. For SMAP L4 DELTA=3hours

  frames_PPT = [] 
  frames_SM = []

  while start <= end:
    d = start.strftime("%Y%m%d%H%M%S")
    # Importing SMAP precipitation data
    img_PPT = Image.open(path_PPT+d+"_ppt.jpeg")
    f_PPT = img_PPT.convert(mode='RGB')
    f_PPT = np.array(f_PPT)
    f_PPT = np.sum(f_PPT, axis=2)
    f_PPT = np.array(f_PPT)
    f_PPT = f_PPT.reshape(f_PPT.shape[0], f_PPT.shape[1], 1)
    frames_PPT.append(f_PPT)
    
    # Importing SMAP soil moisture data
    img_SM = Image.open(path_SM+d+"_sm.jpeg")
    f_SM = img_SM.convert(mode='RGB')
    f_SM = np.array(f_SM)
    f_SM = np.sum(f_SM, axis=2)
    f_SM = np.array(f_SM)
    f_SM = f_SM.reshape(f_SM.shape[0], f_SM.shape[1], 1)
    frames_SM.append(f_SM)

    start += delta

  frames_PPT = np.array(frames_PPT)
  frames_SM = np.array(frames_SM)
  print(f"Shape of frames_PPT is {np.shape(frames_PPT)}")
  print(f"shape of frames_SM is {np.shape(frames_SM)}")

  # Normalization of the SM and PPT images

  test_frames_SM = frames_SM[0:(len(frames_SM)),:,:,:]
  test_frames_PPT = frames_PPT[0:(len(frames_PPT)),:,:,:]

  maxi_SM = np.max(test_frames_SM)
  mini_SM = np.min(test_frames_SM)

  maxi_PPT = np.max(test_frames_PPT)
  mini_PPT = np.min(test_frames_PPT)

  test_frames_SM = (test_frames_SM-mini_SM)/(maxi_SM-mini_SM)
  test_frames_PPT = (test_frames_PPT-mini_PPT)/(maxi_PPT-mini_PPT)

  print(f"Shape of test_frames_SM is {np.shape(test_frames_SM)}")
  print(f"Shape of test_frames_PPT is {np.shape(test_frames_PPT)}")

  window = 3 # Interval, using 6 hours interval(i.e., three data prediction)

  # Appending 3 images to a single variable ---> input to the model data structure

  X_test_SM, X_test_PPT, y_act = [], [], []

  for i in range(0, len(test_frames_PPT), 3):
    X_test_SM.append(test_frames_SM[i:i+window,::,::,::])
    X_test_PPT.append(test_frames_PPT[i:i+window,::,::,::])

  for i in range(0, len(test_frames_SM), 3):
    y_act.append(test_frames_SM[i:i+window,::,::,::])
    
  X_test_SM = np.array(X_test_SM)
  X_test_PPT = np.array(X_test_PPT)
  y_act = np.array(y_act)

  print(f"Shape of X_test_SM is {np.shape(X_test_SM)}")
  print(f"Shape of X_test_PPT is {np.shape(X_test_PPT)}")
  print(f"Shape of y_act is {np.shape(y_act)}")

  # Loading trained model

  model = tf.keras.models.load_model('/content/gdrive/My Drive/Archana/Colab_Notebooks/Models/global_model_v1.h5')

  # Prediction

  avg_RMSE = []
  avg_bias = []
  
  # a and b are just dummy variables, since np.newaxis did not work in a single line
  # xi's are the inputs, other inputs can be added

  a = X_test_SM[0,::,::,::,::] 
  x1 = a[np.newaxis,::,::,::,::] # x1 is the input - SM

  dfs = []

  # Predicting for three days
  for i in range(1,9):
    b = X_test_PPT[i-1,::,::,::,::]
    x2 = b[np.newaxis,::,::,::,::] # x2 is the input - PPT
    
    # Predicting in three hour intervals, model trained that way
    for j in range(0,3):
      predictions = []
      n_iter = 2 # Number of iterations for prediction, using dropout layer to predict the mean and other statistic
      for t in range(n_iter):
        pred = model.predict([x1, x2])
        y_pred = pred[::,j,::,::,::].flatten()
        predictions.append(y_pred)
      
      prediction_df = pd.DataFrame()
      pred_array = np.array(predictions)
      prediction_df['mean'] = pred_array.mean(axis=0).reshape(-1,)
      prediction_df['std'] = pred_array.std(axis=0).reshape(-1,)
      prediction_df['var'] = pred_array.var(axis=0).reshape(-1,)

      y_true = y_act[i,j,::,::,::].flatten()

      pred_mean = prediction_df['mean']
      std = prediction_df['std']
      var = prediction_df['var']


      dict_tosave = {"y_pred":pred_mean,
                    "Std_dev":std,
                    "Var":var}
      
      df_tosave = pd.DataFrame(dict_tosave)
      dfs.append(df_tosave)
      
      
      
      avg_RMSE.append(math.sqrt(mean_squared_error(y_true, pred_mean)))
      avg_bias.append(abs(statistics.mean(pred_mean)-statistics.mean(y_true)))
      
      
    a = pred[0,::,::,::,::]
    x1 = a[np.newaxis,::,::,::,::]

  # Saving 3rd day predictions(in real time today)
  pred_start = datetime.datetime(2020,10,i_date+3,1,30,0)
  pred_end = datetime.datetime(2020,10,i_date+3,22,30,0)
  k = np.shape(dfs)[0]-1
  
  while pred_end >= pred_start:
    pred_d = pred_end.strftime("%Y%m%d%H%M%S")
    dfs[k].to_csv(path_OP+pred_d+".csv",index=False)
    k = k-1
    pred_end -= delta

  RMSE = RMSE+avg_RMSE[-8:]
  bias = bias+avg_bias[-8:]

  print("ONE DAY DONE!")

stat_dict = {"RMSE": RMSE,
             "Bias": bias}
stat_df = pd.DataFrame(stat_dict)
stat_df.to_csv(path_OP+"Statistics.csv",index=False)